In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [156]:
from plotly import __version__
from plotly import tools
import plotly.dashboard_objs as dashboard
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly.graph_objs import *
init_notebook_mode(connected=True)

In [3]:
import os
import pandas as pd
import json
import folium
import re
import shutil
import json
from colour import Color
from numpy import interp
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
from folium import FeatureGroup, LayerControl, Map, Marker
from folium.plugins.measure_control import MeasureControl
from folium import plugins
import branca
import matplotlib.image as mpimg
from IPython.display import IFrame
import requests
import matplotlib.pyplot as plt

In [4]:
def buildJsonAPI(url):
    restcountries = {}
    nametoid = {}
    response = requests.get(url)
    for json_country in response.json():
        try:
            code = int(json_country['numericCode'])
            restcountries[code] = {}
            restcountries[code]['name'] = json_country['name']
            restcountries[code]['latlng'] = json_country['latlng']
            restcountries[code]['name'] = json_country['name']
            restcountries[code]['alpha3Code'] = json_country['alpha3Code']
            nametoid[json_country['name']] = code
        except:
            print(json_country['name'] + " has problem with numeric code " + str(json_country['numericCode']))
    return restcountries, nametoid

In [5]:
def parseCountries(active_flows):
    cols=[i for i in actives_flows.columns if i not in ["Country_name"]]
    for col in cols:
        actives_flows[col]=pd.to_numeric(actives_flows[col],errors='coerce')
    return active_flows

In [6]:
def addDetails(actives_flows_by_country2):
    actives_flows_by_country = actives_flows_by_country2.copy()
    for i, row in actives_flows_by_country.iterrows():
        actives_flows_by_country.loc[i, 'CODE'] = restcountries[int(row['Country'])]['alpha3Code']
        actives_flows_by_country.loc[i, 'Name'] = restcountries[int(row['Country'])]['name']
    return actives_flows_by_country

In [7]:
def getNewBoundedMap():
    min_lon, max_lon = -180, 180
    min_lat, max_lat = -90, 90
    return folium.Map([46.8,8], 
                       tiles='Stamen Terrain',
                       zoom_start=1.5, 
                       max_bounds=True,
                       min_lat=min_lat,
                       max_lat=max_lat,
                       min_lon=min_lon,
                       max_lon=max_lon)

In [8]:
def addCloropleth(dataframe, map_, feature, legend_name, key, scale_color,json_data,object_):
    map_.choropleth(geo_data=json_data,
                    data=dataframe,
                    columns=[key, feature],
                    key_on='feature.id',
                    threshold_scale=[100, 500, 1000, 5000, 10000, 40000],
                    fill_color=scale_color,
                    fill_opacity=0.7, 
                    line_opacity=0.5,
                    highlight=True,
                    legend_name=legend_name, 
                    topojson = object_)

In [9]:
def getData(dataframe, year):
    data = [ dict(
        type = 'choropleth',
        locations = dataframe['CODE'],
        z = dataframe[str(year)],
        text = dataframe['Name'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '$',
            title = 'Active offshores per country<br>In ' + str(year)),
      ) ]
    return data

In [208]:
def getLayout():
    layout = dict(
        title = 'Active offshores per country <br> <br> Source:\
                <a href="https://www.icij.org/">\
                International Consortium of Investigative Journalists</a>',
        geo = dict(
            projection = dict(
                type = 'Mercator'
            ),
            resolution='50',
        )
    )
    return layout

In [209]:
def visualizeFlowByCountry(country, year):
    name = country
    country_df = actives_flows[actives_flows['Country'] == nametoid[country]]
    total_offshores = 0
    labels = []
    values = []
    flows_path = []
    max_value = country_df[str(year)].max()
    for i, row in country_df.iterrows():
        if not pd.isnull(row['Country']) and not pd.isnull(row['jurisdiction']) and not pd.isnull(row[str(year)]):
            coord = [restcountries[int(row['Country'])]['latlng'], restcountries[int(row['jurisdiction'])]['latlng']]
            total_offshores = total_offshores + row[str(year)]
            labels.append(restcountries[int(row['jurisdiction'])]['name'])
            values.append(row[str(year)])
            flows_path.append(
                    dict(
                        type = 'scattergeo',
                        lon = [ coord[0][1], coord[1][1] ],
                        lat = [ coord[0][0], coord[1][0] ],
                        mode = 'lines',
                        line = dict(
                            width = 1,
                            color = 'red',
                        ),
                        opacity = max(float(row[str(year)] / max_value), 0.3),
                        showlegend = False,
                        name=restcountries[int(row['jurisdiction'])]['name'],
                    )
                )

    fig = dict( data=flows_path + getData(actives_flows_by_country, year), layout=getLayout() )
    iplot( fig, validate=False, filename='d3-world-map.html')
    trace = go.Pie(labels=labels, 
                   values=values,
                  textfont=dict(size=20),)
    iplot([trace], filename='basic_pie_chart.html')
    #data = Data([flows_path + getData(actives_flows_by_country, year), trace])
    #figure = Figure(data=data, layout=getLayout())
    print("Total offshores in " + str(year) + " = " + str(total_offshores))

In [210]:
def fileId_from_url(url_x):
    """Return fileId from a url."""
    raw_fileId = re.findall("~[A-z]+/[0-9]+", url_x)[0][1: ]
    return raw_fileId.replace('/', ':')

## Countries that we want to analyze

In [211]:
countries = [
    'Switzerland', 
    'Hong Kong', 
    'China', 
    'Russian Federation', 
    'United States of America',
    'Bahamas',
    'Luxembourg'
]

In [212]:
url = 'https://restcountries.eu/rest/v2/'

In [213]:
actives_path = r'csv/cash_flows_actives.csv'
world_topojson_path = r'json/world-50m.json'

In [214]:
restcountries, nametoid = buildJsonAPI(url)

Republic of Kosovo has problem with numeric code None


In [215]:
actives_flows = pd.read_csv(actives_path,low_memory=False)
world_topojson = json.load(open(world_topojson_path))

In [216]:
actives_flows = parseCountries(actives_flows)

In [217]:
actives_flows_by_country = actives_flows.groupby('Country').sum().reset_index()
actives_flows_by_country = addDetails(actives_flows_by_country)

In [218]:
interactive(visualizeFlowByCountry, country=countries, year=(1950, 2016, 1))

A Jupyter Widget

## Observations

### After the leaks
We can state that after the leakage, the majority of the most involved countries stopped investing in the jurisdiction of Panama, which in the last years has encountered an increasing number of inactivations.
On the countrary, the British overseas territories has encountered an increasing number of incorporations.

The #PanamaPapers scandals made a lot of countries close their offshore.
This lead us to investigate in which are the factors that mostly concurred in this reaction, but what are the differences between each jurisdiction.

We will, in parallel with the 1 Research Question, investigate the differences between the most important jurisdiction. To this purpose we will use the **FINANCIAL SECRECY INDEX**